### Customer data ingestion procedure table creation

In [ ]:
# Obtain access to Datalake en 2 Storage

spark.conf.set(
    "fs.azure.account.key.fewdatalakegen2.dfs.core.windows.net",
    dbutils.secrets.get(scope="Few-Bi-Scope", key="fewdatalakegen2key"))

In [ ]:
# Import packages

import datetime
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import json
from dateutil import parser, tz
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, TimestampType, ArrayType, DateType
from pyspark.sql.functions import col, concat, current_timestamp, to_date, explode, to_timestamp, to_utc_timestamp, from_utc_timestamp, size, when


In [ ]:
# List tenants for connections

tenants = ['12th-gate-napa-valley', '1967-cellars-napa-valley', '1883-reserve-napa-valley',
       '510-pearl-napa-valley', '90-broad-napa-valley',
       'black-and-gold-napa-valley', 'bold-stripes-napa-valley',
       'brownstown-napa-valley', 'ever-blooming-napa-valley',
       'excelsior-napa-valley', 'green-19-napa-valley', 'fairwinds-estate-winery',
       'first-moon-napa-valley', 'foolish-club-napa-valley',
       'founders-rock-napa-valley', 'four-points-napa-valley',
       'golden-anchor-napa-valley', 'great-day-napa-valley',
       'great-minds-napa-valley', 'horns-tail-napa-valley',
       'in-hoc-napa-valley', 'killarney-napa-valley',
       'landfall-napa-valley', 'les-soeurs-fideles-napa-valley','long-gray-line-napa-valley','maverick-moon-napa-valley',
       'mountain-born-napa-valley', 'mystic-circle-napa-valley',
       'nineteen-seventeen-napa-valley', 'obbligato-napa-valley',
       'one-heart-napa-valley', 'parasol-napa-valley', 'perpetual-napa-valley',
       'phiphika-napa-valley', 'prowling-vineyards-napa-valley',
       'radiant-napa-valley', 'rattlesnake-napa-valley',
       'red-door-napa-valley', 'ruby-points-napa-valley',
       'seeing-eye-napa-valley', 'spindletop-napa-valley',
       'steel-curtain-napa-valley', 'tg-napa-valley', 'the-fleur-napa-valley',
       'the-ivy-napa-valley', 'three-degrees-napa-valley',
       'three-strings-napa-valley', 'tower-napa-valley',
       'true-north-napa-valley', 'unmatched-napa-valley',
       'vine-04-napa-valley', 'winning-drive-napa-valley']

In [ ]:
#tenants = ['spindletop-napa-valley', 'first-moon-napa-valley', 'one-heart-napa-valley']

In [ ]:
# Instantiate empty array

customers = []

# Loop over tenants and assign tenantcode and brand

for t in tenants: 
        if t == 'fairwinds-estate-winery':
            tenantcode = 'FEW'
            brand = "Fairwinds Estate Winery"
        elif t == '12th-gate-napa-valley':
            tenantcode = '12G'
            brand = "12th Gate"
        elif t == '510-pearl-napa-valley':
            tenantcode = '5P'
            brand = '510 Pearl'
        elif t == '90-broad-napa-valley':
            tenantcode = '90B'
            brand = "90 Broad"
        elif t == 'black-and-gold-napa-valley':
            tenantcode = 'BG'
            brand = 'Black & Gold'
        elif t == 'brownstown-napa-valley':
            tenantcode = 'BT'
            brand = 'Brownstown'
        elif t == 'excelsior-napa-valley':
            tenantcode = 'EX'
            brand = 'Excelsior'
        elif t == 'first-moon-napa-valley':
            tenantcode = 'FM'
            brand = 'First Moon'
        elif t == 'foolish-club-napa-valley':
            tenantcode = 'FOO'
            brand = 'Foolish Club'
        elif t == 'founders-rock-napa-valley':
            tenantcode = 'FR'
            brand = 'Founders Rock'
        elif t == 'four-points-napa-valley':
            tenantcode = 'FP'
            brand = 'Four Points'
        elif t == 'golden-anchor-napa-valley':
            tenantcode = 'GA'
            brand = 'Golden Anchor'
        elif t == 'great-day-napa-valley':
            tenantcode = 'GD'
            brand = 'Great Day'
        elif t == 'in-hoc-napa-valley':
            tenantcode = 'IH'
            brand = 'In Hoc'
        elif t == 'killarney-napa-valley':
            tenantcode = 'KI'
            brand = 'Killarney'
        elif t == 'maverick-moon-napa-valley':
            tenantcode = 'MM'
            brand = 'Maverick Moon'
        elif t == 'mountain-born-napa-valley':
            tenantcode = 'MB'
            brand = 'Mountain Born'
        elif t == 'mystic-circle-napa-valley':
            tenantcode = 'MC'
            brand = 'Mystic Circle'
        elif t == 'nineteen-seventeen-napa-valley':
            tenantcode = 'NS'
            brand = 'Nineteen Seventeen'
        elif t == 'one-heart-napa-valley':
            tenantcode = 'OH'
            brand = 'One Heart'
        elif t == 'parasol-napa-valley':
            tenantcode = 'PR'
            brand = 'Parasol'
        elif t == 'phiphika-napa-valley':
            tenantcode = 'PHI'
            brand = 'PhiPhiKa'
        elif t == 'prowling-vineyards-napa-valley':
            tenantcode = 'PV'
            brand = 'Prowling Vineyards'
        elif t == 'radiant-napa-valley':
            tenantcode = 'RA'
            brand = 'Radiant'
        elif t == 'rattlesnake-napa-valley':
            tenantcode = 'RS'
            brand = 'Rattlesnake'
        elif t == 'red-door-napa-valley':
            tenantcode = 'RD'
            brand = 'Red Door'
        elif t == 'rituelle-napa-valley':
            tenantcode = 'RT'
            brand = 'Rituelle'
        elif t == 'ruby-points-napa-valley':
            tenantcode = 'RP'
            brand = 'Ruby Points'
        elif t == 'seeing-eye-napa-valley':
            tenantcode = 'SE'
            brand = 'Seeing Eye'
        elif t == 'spindletop-napa-valley':
            tenantcode = 'ST'
            brand = 'Spindletop'
        elif t == 'tg-napa-valley':
            tenantcode = 'TG'
            brand = 'TG'
        elif t == 'the-ivy-napa-valley':
            tenantcode = 'IV'
            brand = 'The Ivy'
        elif t == 'three-degrees-napa-valley':
            tenantcode = 'TD'
            brand = 'Three Degrees'
        elif t == 'three-strings-napa-valley':
            tenantcode = 'TS'
            brand = 'Three Strings'
        elif t == 'tower-napa-valley':
            tenantcode = 'TW'
            brand = 'Tower'
        elif t == 'true-north-napa-valley':
            tenantcode = 'TN'
            brand = 'True North'
        elif t == 'unmatched-napa-valley':
            tenantcode = 'UN'
            brand = 'Unmatched'
        elif t == 'winning-drive-napa-valley':
            tenantcode = 'WD'
            brand = 'Winning Drive'
        elif t == 'landfall-napa-valley':
            tenantcode = 'LF'
            brand = 'Landfall'
        elif t == 'obbligato-napa-valley':
            tenantcode = 'OB'
            brand = 'Obbligato'
        elif t == 'ever-blooming-napa-valley':
            tenantcode = 'EB'
            brand ='Ever Blooming'
        elif t == 'great-minds-napa-valley':
            tenantcode = 'GM'
            brand = 'Great Minds'
        elif t == 'steel-curtain-napa-valley':
            tenantcode = 'SC'
            brand = 'Steel Curtain'
        elif t == '1967-cellars-napa-valley':
            tenantcode = 'CL'
            brand = '1967 Cellars'
        elif t == 'horns-tail-napa-valley':
            tenantcode = 'HT'
            brand = 'Horns & Tail'
        elif t == 'vine-04-napa-valley':
            tenantcode = 'V4'
            brand = 'Vine 04'
        elif t == 'bold-stripes-napa-valley':
            tenantcode = 'BS'
            brand = 'Bold Stripes'
        elif t == 'perpetual-napa-valley':
            tenantcode = 'PER'
            brand = 'Perpetual'
        elif t == 'green-19-napa-valley':
            tenantcode = 'WS'
            brand = 'Winning Standard'
        elif t == '1883-reserve-napa-valley':
            tenantcode = 'YS'
            brand = 'Yellowstone'
        elif t == 'long-gray-line-napa-valley':
            tenantcode = 'LG'
            brand = 'Long Gray Line'
        elif t == 'les-soeurs-fideles-napa-valley':
            tenantcode = 'LS'
            brand = 'Les Soeurs Fideles'
        elif t == 'the-fleur-napa-valley':
            tenantcode = 'TF'
            brand = 'The Fleur'
        else:
            tenantcode = 'NaN'

        # Instantiate cursor

        params = {'cursor': 'start'}

        upDate = datetime.datetime.today()
        while True:
            url = 'https://api.commerce7.com/v1/customer'
            response = requests.get(url, auth = HTTPBasicAuth('wilfredo@fewinery.com', 'Conexion@4$'), 
                                    headers = {"tenant":t, "Content-Type":"application/json"},
                                    params=params)

            res = response.json()
            #logging.info(res)
            
            for row in res['customers']:
                row['tenant'] = t
                row['tenantCode'] = tenantcode
                row['brand'] = brand
                #row['uniqueOrderNumber'] = tenantcode + str(row['orderNumber'])
                #row['timesOnHold'] = len(row['onHolds'])
                #row['onHold'] = None if len(row['onHolds']) == 0 else row['onHolds'][0]
                #row['cancelDate'] = None if row['cancelDate'] is None else datetime.datetime.strptime(row['cancelDate'], '%Y-%m-%dT%H:%M:%S.%fZ')
                #row['updateDate'] = upDate
                #row['membershipYear'] = parser.parse(row['createdAt']).strftime("%Y")#upDate.strftime("%Y")
                #row.pop('giftMessage')
                #row['transType'] = 'Deposit' if (len(row['items']) == 1 and row['items']['sku'] == "50Deposit") else row['purchaseType']
                customers.append(row)

            if (cursor := res.get('cursor')):
                params['cursor'] = cursor
                #print(len(res['orders']), tenantorders[-1]['orderNumber'], tenantorders[-1]['tenantCode'])
            else:
                #print(len(res['orders']), tenantorders[-1]['orderNumber'], tenantorders[-1]['tenantCode'])
                break
        print(response, t, datetime.datetime.now(), len(customers), upDate)

<Response [200]> 12th-gate-napa-valley 2023-01-25 19:10:33.933118 542 2023-01-25 19:10:28.404531
<Response [200]> 1967-cellars-napa-valley 2023-01-25 19:10:34.464608 575 2023-01-25 19:10:33.933299
<Response [200]> 1883-reserve-napa-valley 2023-01-25 19:10:39.552216 1263 2023-01-25 19:10:34.464768
<Response [200]> 510-pearl-napa-valley 2023-01-25 19:10:40.882614 1396 2023-01-25 19:10:39.552370
<Response [200]> 90-broad-napa-valley 2023-01-25 19:10:42.020823 1589 2023-01-25 19:10:40.882766
<Response [200]> black-and-gold-napa-valley 2023-01-25 19:10:45.438461 2134 2023-01-25 19:10:42.021000
<Response [200]> bold-stripes-napa-valley 2023-01-25 19:10:47.490168 2409 2023-01-25 19:10:45.438623
<Response [200]> brownstown-napa-valley 2023-01-25 19:10:49.131482 2592 2023-01-25 19:10:47.490319
<Response [200]> ever-blooming-napa-valley 2023-01-25 19:10:51.835996 2899 2023-01-25 19:10:49.131639
<Response [200]> excelsior-napa-valley 2023-01-25 19:10:54.837127 3391 2023-01-25 19:10:51.836168
<Res

In [ ]:
customers[0]

Out[6]: {'id': '016fd112-7a62-4a12-ac77-ee8f646abf7a',
 'avatar': None,
 'honorific': None,
 'firstName': 'Aaron',
 'lastName': 'Pead',
 'birthDate': '1985-04-07',
 'city': 'Santa Rosa',
 'stateCode': 'CA',
 'zipCode': '95403',
 'countryCode': 'US',
 'emailMarketingStatus': None,
 'lastActivityDate': '2022-04-16T07:02:57.080Z',
 'metaData': {'seniority-number': 140,
  'gift-membership': False,
  'wine-preference': 'Red',
  'are-you-a-member-of-kappa-sigma-': True,
  'affiliation': '',
  'university-name': 'Idaho State University',
  'pledge-year': 2013,
  'was-your-partner-in-a-greek-organization-': 'No',
  'greek-organization': ''},
 'appData': None,
 'appSync': None,
 'flags': [],
 'notifications': [],
 'createdAt': '2021-02-13T17:30:12.490Z',
 'updatedAt': '2022-09-24T07:07:50.940Z',
 'orderInformation': {'currentWebCartId': None,
  'lastOrderId': 'aac2a908-16f8-45ac-8f04-26459edc2482',
  'lastOrderDate': '2022-09-24T07:07:49.766Z',
  'orderCount': 5,
  'lifetimeValue': 140496,
  'l

In [ ]:
print(*customers[0]['metaData'].keys(), sep='\n')

seniority-number
gift-membership
wine-preference
are-you-a-member-of-kappa-sigma-
affiliation
university-name
pledge-year
was-your-partner-in-a-greek-organization-
greek-organization


In [ ]:
metaDataSchema = StructType(fields = [StructField("seniority-number", StringType(), True), 
                                        StructField("gift-membership", StringType(), True),
                                        StructField("wine-preference", StringType(), True),
                                        StructField("are-you-a-member-of-kappa-sigma-", StringType(), True),
                                        StructField("affiliation", StringType(), True),
                                        StructField("university-name", StringType(), True),
                                        StructField("pledge-year", StringType(), True),
                                        StructField("was-your-partner-in-a-greek-organization-", StringType(), True),
                                        StructField("greek-organization", StringType(), True)])

In [ ]:
clubSchema = StructType(fields = [StructField("clubId", StringType(), True),
                                    StructField("clubTitle", StringType(), True),
                                    StructField("cancelDate", StringType(), True),
                                    StructField("signupDate", StringType(), True),
                                    StructField("clubMembershipId", StringType(), True)])

In [ ]:
prodSchema = StructType(fields = [StructField("sku", StringType(), True),
                                    StructField("image", StringType(), True),
                                    StructField("price", IntegerType(), True),
                                    StructField("title", StringType(), True),
                                    StructField("quantity", IntegerType(), True),
                                    StructField("productId", StringType(), True),
                                    StructField("wineProperties", StringType(), True)])

productSchema = StructType(fields = [StructField("product", prodSchema, True),
                                        StructField("purchaseDate", StringType(), True)])

In [ ]:
groupSchema = StructType(fields = [StructField("id", StringType(), True), 
                                    StructField("title", StringType(), True), 
                                    StructField("objectType", StringType(), True), 
                                    StructField("type", StringType(), True), 
                                    StructField("appliesToCondition", StringType(), True), 
                                    StructField("createdAt", StringType(), True), 
                                    StructField("updatedAt", StringType(), True), 
                                    StructField("backPopulate", StringType(), True)])

In [ ]:
tagSchema = StructType(fields = [StructField("id", StringType(), True),
                                    StructField("title", StringType(), True),
                                    StructField("objectType", StringType(), True),
                                    StructField("type", StringType(), True),
                                    StructField("appliesToCondition", StringType(), True),
                                    StructField("createdAt", StringType(), True),
                                    StructField("updatedAt", StringType(), True),
                                    StructField("backPopulate", StringType(), True)])

In [ ]:
phoneSchema = StructType(fields = [StructField("id", StringType(), True),
                                    StructField("phone", StringType(), True)])

emailSchema = StructType(fields = [StructField("id", StringType(), True),
                                    StructField("email", StringType(), True),
                                    StructField("status", StringType(), True)])

In [ ]:
orderInfoSchema = StructType(fields = [StructField("currentWebCartId", StringType(), True),
                                        StructField("lastOrderId", StringType(), True),
                                        StructField("lastOrderDate", StringType(), True),
                                        StructField("orderCount", StringType(), True),
                                        StructField("lifetimeValue", StringType(), True),
                                        StructField("lifetimeValueSeedAmount", StringType(), True),
                                        StructField("yearlyValue", StringType(), True),
                                        StructField("rank", StringType(), True),
                                        StructField("rankTrend", StringType(), True),
                                        StructField("grossProfit", StringType(), True),
                                        StructField("acquisitionChannel", StringType(), True),
                                        StructField("currentClubTitle", StringType(), True),
                                        StructField("daysInCurrentClub", StringType(), True),
                                        StructField("daysInClub", StringType(), True)])

In [ ]:
customerSchema = StructType(fields = [StructField("id", StringType(), True),
                                        StructField("avatar", StringType(), True),
                                        StructField("honorific", StringType(), True),
                                        StructField("firstName", StringType(), True),
                                        StructField("lastName", StringType(), True),
                                        StructField("birthDate", StringType(), True),
                                        StructField("city", StringType(), True),
                                        StructField("stateCode", StringType(), True),
                                        StructField("zipCode", StringType(), True),
                                        StructField("countryCode", StringType(), True),
                                        StructField("emailMarketingStatus", StringType(), True),
                                        StructField("lastActivityDate", StringType(), True),
                                        StructField("metaData", metaDataSchema, True),
                                        StructField("appData", StringType(), True),
                                        StructField("appSync", StringType(), True),
                                        StructField("flags", StringType(), True),
                                        StructField("notifications", StringType(), True),
                                        StructField("createdAt", StringType(), True),
                                        StructField("updatedAt", StringType(), True),
                                        StructField("orderInformation", orderInfoSchema, True),
                                        StructField("emails", ArrayType(emailSchema), True),
                                        StructField("phones", ArrayType(phoneSchema), True),
                                        StructField("tags", ArrayType(tagSchema), True),
                                        StructField("groups", ArrayType(groupSchema), True),
                                        StructField("clubs", ArrayType(clubSchema), True),
                                        StructField("products", ArrayType(productSchema), True),
                                        StructField("hasAccount", StringType(), True),
                                        StructField("loginActivity", StringType(), True),
                                        StructField("tenant", StringType(), True),
                                        StructField("tenantCode", StringType(), True),
                                        StructField("brand", StringType(), True)])

sparkdf = spark.createDataFrame(customers, customerSchema)

In [ ]:
display(sparkdf)

id avatar honorific firstName lastName birthDate city stateCode zipCode countryCode emailMarketingStatus lastActivityDate metaData appData appSync flags notifications createdAt updatedAt orderInformation emails phones tags groups clubs products hasAccount loginActivity tenant tenantCode brand 016fd112-7a62-4a12-ac77-ee8f646abf7a null null Aaron Pead 1985-04-07 Santa Rosa CA 95403 US null 2022-04-16T07:02:57.080Z List(140, false, Red, true, , Idaho State University, 2013, No, ) null null [] [] 2021-02-13T17:30:12.490Z 2022-09-24T07:07:50.940Z List(null, aac2a908-16f8-45ac-8f04-26459edc2482, 2022-09-24T07:07:49.766Z, 5, 140496, 0, {2022=75477, 2021=65019}, 0, null, 116151, null, 1869 Club, 696, 696) List(List(2fe49f94-2e9b-4df6-a462-ea801fec3ad4, peadaaro@isu.edu, Ok)) List(List(482f7058-197b-4ea1-addb-1786758b5d67, +17073089857)) List(List(c06c0d54-fef6-4bb8-8607-8250e0683964, Deposits, Customer, Dynamic, One or more conditions, 2021-03-23T00:28:39.653Z, 2021-03-23T00:28:41.450Z, {cursor=fce5ba9f-874f-4ead-a576-ba64eae4b56e, totalPages=7, currentPage=7, processDate=2021-03-23T00:28:41.450Z, status=Completed})) List(List(c06c0d54-fef6-4bb8-8607-8250e0683964, Deposits, Customer, Dynamic, One or more conditions, 2021-03-23T00:28:39.653Z, 2021-03-23T00:28:41.450Z, {cursor=fce5ba9f-874f-4ead-a576-ba64eae4b56e, totalPages=7, currentPage=7, processDate=2021-03-23T00:28:41.450Z, status=Completed})) List(List(bcb60376-ee39-4421-aeb3-7a308b38af7b, 1869 Club, null, 2021-02-13T17:30:13.789Z, 97bbd332-7070-4194-a79c-4089487473e8)) List(List(List(CG-12GPN21RR, https://images.commerce7.com/12th-gate-napa-valley/images/original/kappasigma_12thgate_sajackson_pinotnoir-1663109629926.png, 5500, S. A. Jackson - 2021 Pinot Noir – Russian River Valley (Member Price: $55.00), 2, 02f53218-4280-45f9-b1b6-2927b5b5cee5, {countryCode=US, vintage=2021, varietal=Pinot Noir, appellation=Russian River Valley, region=California, type=Red}), 2022-09-24T07:07:49.766Z), List(List(NW-12G6WB, https://images.commerce7.com/12th-gate-napa-valley/images/original/12thgatewoodenbox_211x143px-1617399480775.png, 0, Wooden Crate, 1, e4544b99-0f80-4600-a06d-ac39dde88478, null), 2022-09-24T07:07:49.766Z), List(List(NW-12GWKF, https://images.commerce7.com/12th-gate-napa-valley/images/original/12thgate_winekey-1617399558453.png, 0, 12th Gate Wine Key, 1, f79e1183-0c49-42e2-9a95-db1753b134f6, null), 2022-09-24T07:07:49.766Z), List(List(CG-12GCS20NV, https://images.commerce7.com/12th-gate-napa-valley/images/original/kappasigma_12thgate_crescent_cabernetsauvignon-1663109611448.png, 7500, Crescent - 2020 Cabernet Sauvignon - Napa Valley (Member Price: $75.00), 2, 8ff4f7be-183c-402e-aaf2-7806756cad51, {countryCode=US, vintage=2020, varietal=Cabernet Sauvignon, appellation=Napa Valley, region=California, type=Red}), 2022-09-24T07:07:49.766Z), List(List(CG-12GWW21NV, https://images.commerce7.com/12th-gate-napa-valley/images/original/kappasigma_12thgate_halo_cuveeblanc-1663109643954.png, 3500, Halo - 2021 Cuvée Blanc – Napa Valley (Member Price: $35.00), 1, 8c52f064-28ec-43b7-b14a-a47f780ea1c7, {countryCode=US, vintage=2021, varietal=Blend, appellation=Napa Valley, region=California, type=White}), 2022-09-24T07:07:49.766Z), List(List(12GATEWINEKEYF, https://images.commerce7.com/12th-gate-napa-valley/images/original/12thgate_winekey-1617399558453.png, 0, 12th Gate Wine Key, 1, f79e1183-0c49-42e2-9a95-db1753b134f6, null), 2022-04-16T07:02:55.895Z), List(List(2018FEZINFANDEL, https://images.commerce7.com/12th-gate-napa-valley/images/original/fw_zinfandel_mainstay_2018-1634067524932.png, 4250, Fairwinds - Mainstay - 2018 Zinfandel - Mount Veeder, Napa Valley, 1, 38b13e6d-2903-432c-8052-e2c5ed2ff716, {countryCode=US, vintage=2018, varietal=Zinfandel, appellation=Mt. Veeder, region=California, type=Red}), 2022-04-16T07:02:55.895Z), List(List(PC-FEWSP22VB, https://images.commerce7.com/12th-gate-napa-valley/images/original/verticalbundle_fairwinds-1648583703080.png, 28900, Fairwinds Vertical 3

In [ ]:
sparkdf = sparkdf.withColumn("lastActivityDateT", from_utc_timestamp(col("lastActivityDate"), "America/El_Salvador"))\
                    .withColumn("lastActivityDate", to_date(from_utc_timestamp(col("lastActivityDate"),"America/El_Salvador")))\
                    .withColumn("orderCount", col("orderInformation.orderCount"))\
                    .withColumn("lifetimeValue", col("orderInformation.lifetimeValue"))\
                    .withColumn("daysInClub", col("orderInformation.daysInClub"))\
                    .withColumn("currentClubTitle", col("orderInformation.currentClubTitle"))\
                    .withColumn("email", col("emails.email").getItem(0))\
                    .withColumn("phone", col("phones.phone").getItem(0))\
                    .withColumn("cancelDate", col("clubs.cancelDate").getItem(size(col("clubs.cancelDate"))-1))\
                    .withColumn("createdAtT", from_utc_timestamp(col("createdAt"), "America/El_Salvador"))\
                    .withColumn("createdAt", to_date(from_utc_timestamp(col("createdAt"), "America/El_Salvador")))\
                    .withColumn("updatedAtT", from_utc_timestamp(col("updatedAt"), "America/El_Salvador"))\
                    .withColumn("updatedAt", to_date(from_utc_timestamp(col("updatedAt"), "America/El_Salvador")))\
                    .withColumn("cancelDate", from_utc_timestamp(col("cancelDate"), "America/El_Salvador"))\
                    .withColumn("cancelDateT", to_date(from_utc_timestamp(col("cancelDate"), "America/El_Salvador")))\
                    .withColumn("canceled", when(col("cancelDate").isNotNull(), True).otherwise(False))

/databricks/spark/python/pyspark/sql/column.py:423: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [ ]:
sparkdf.printSchema()

root
 |-- id: string (nullable = true)
 |-- avatar: string (nullable = true)
 |-- honorific: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- city: string (nullable = true)
 |-- stateCode: string (nullable = true)
 |-- zipCode: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- emailMarketingStatus: string (nullable = true)
 |-- lastActivityDate: date (nullable = true)
 |-- metaData: struct (nullable = true)
 |    |-- seniority-number: string (nullable = true)
 |    |-- gift-membership: string (nullable = true)
 |    |-- wine-preference: string (nullable = true)
 |    |-- are-you-a-member-of-kappa-sigma-: string (nullable = true)
 |    |-- affiliation: string (nullable = true)
 |    |-- university-name: string (nullable = true)
 |    |-- pledge-year: string (nullable = true)
 |    |-- was-your-partner-in-a-greek-organization-: string (nullable = true)
 |    |-- g

In [ ]:
sparkdf = sparkdf.select("brand", "tenantCode", "tenant", 
                         "id", "firstName", "lastName", "birthDate", "city", "stateCode", "zipCode", "countryCode", 
                         "createdAt", "updatedAt",
                         "cancelDate", "canceled", "lastActivityDate",
                         "currentClubTitle", "orderCount", "lifetimeValue", "daysInClub", 
                         "email", "phone" 
                        )               


In [ ]:
display(sparkdf.filter(sparkdf.orderCount == 0))

brand,tenantCode,tenant,id,firstName,lastName,birthDate,city,stateCode,zipCode,countryCode,createdAt,updatedAt,cancelDate,canceled,lastActivityDate,currentClubTitle,orderCount,lifetimeValue,daysInClub,email,phone
12th Gate,12G,12th-gate-napa-valley,01faaf40-61ed-44c1-b064-bd13ff20f996,Chris,Yamamura,1981-04-22,San Mateo,CA,94401,US,2021-02-19,2021-02-19,null,false,2021-02-19,null,0,0,null,chris.yamamura@gmail.com,+14087060044
12th Gate,12G,12th-gate-napa-valley,04ab3ca0-177b-48b9-8257-d236a6cecb1b,Conner,Akins,2000-03-20,Chula Vista,CA,91914,US,2022-05-24,2022-05-24,null,false,2022-05-24,null,0,0,null,theconnerakins20@gmail.com,+16198324302
12th Gate,12G,12th-gate-napa-valley,067a7930-694f-45bc-97ac-27a50724ea19,Georgio,Zikantas,1995-07-28,Windsor,ON,N8R1M6,CA,2022-04-20,2022-05-02,null,false,2022-04-20,null,0,0,null,zikantasg@gmail.com,+16474127703
12th Gate,12G,12th-gate-napa-valley,08ddd8b8-c487-40cb-ba2a-c782a9028546,Shant,Vartan,2001-03-24,North Hills,CA,91343,US,2022-04-05,2022-04-05,null,false,2022-04-05,null,0,0,null,shant.vartan@live.com,+18186198287
12th Gate,12G,12th-gate-napa-valley,167b2d80-600c-4d5e-b8b2-d7fb74ff9aae,Ricardo,Parraga,1977-09-29,Calistoga,CA,94515,US,2021-07-01,2022-04-25,null,false,2021-07-01,null,0,0,null,rparraga@osinearshore.com,+14154449927
12th Gate,12G,12th-gate-napa-valley,191728fd-6d3e-46c6-9b9f-4ef8f511083d,Jayden,Reed-Williams,2000-12-08,New Castle,PA,16105,US,2022-09-11,2022-09-11,null,false,2022-09-11,null,0,0,null,houndpound63@outlook.com,+17246513791
12th Gate,12G,12th-gate-napa-valley,1d8ea1f1-b43b-4890-b51d-6423a47b272d,Brian,Whelehan,1989-01-02,Richmond Hill,GA,31324,US,2022-08-24,2022-08-24,null,false,2022-08-24,null,0,0,null,brianwhelehan@yahoo.com,+13217492690
12th Gate,12G,12th-gate-napa-valley,21485f12-365f-49de-b78f-f3e00383f324,Jack,Liddle,1938-02-15,Vero Beach,FL,32966,US,2021-11-01,2021-11-01,null,false,2021-11-01,null,0,0,null,jackliddle252@gmail.com,+17724943199
12th Gate,12G,12th-gate-napa-valley,2d53ce89-9e4c-454e-98b0-fddb00cea253,Todd,Dugas,1992-04-17,Lafayette,LA,70508,US,2022-11-23,2022-11-23,null,false,2022-11-23,null,0,0,null,todddugas10@gmail.com,+13373293068
12th Gate,12G,12th-gate-napa-valley,33348cf5-7215-4c20-a873-f08d4f4a2c65,Cosy,Perham,1970-07-07,Walnut Creek,CA,94598,US,2021-03-21,2021-03-21,null,false,2021-03-21,null,0,0,null,cosyli@gmail.com,+19253512679


In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS commerce7

In [ ]:
sparkdf.write.option("overwriteSchema", "true").format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("commerce7.memberships")

In [ ]:
sparkdf.write.option("overwriteSchema", "true").format("delta").mode("overwrite").save("abfss://processed@fewdatalakegen2.dfs.core.windows.net/C7Memberships/c7memberships")

#df1.to_delta("abfss://processed@fewdatalakegen2.dfs.core.windows.net/C7Orders", partitions_cols=['brand'])

In [ ]:
# Save the raw json to DataLake
for x in memberships:
    x['updateDate'] = str(x['updateDate'])

#jsondata = json.dumps(memberships)

#dbutils.fs.put('abfss://processed@fewdatalakegen2.dfs.core.windows.net/C7Memberships/C7Memberships.json', jsondata, True)

In [ ]:
# Save the raw json to DataLake

jsondata = json.dumps(memberships)

#Check if month is first day

today= datetime.datetime.today()

yesterday = today.strftime("%Y%m%d")

end = today - datetime.timedelta(days= 1)

endofmonth = end.strftime("%Y%m%d")


dbutils.fs.put('abfss://raw@fewdatalakegen2.dfs.core.windows.net/C7Memberships/C7Memberships.json', jsondata, True)

#upDate.strftime("%Y%m%d")

Wrote 99197003 bytes.
Out[21]: True

In [ ]:
if today.strftime("%d") == "01":

    dbutils.fs.put('abfss://raw@fewdatalakegen2.dfs.core.windows.net/C7Memberships/C7Memberships.json', jsondata, True)
    
    dbutils.fs.put(f'abfss://raw@fewdatalakegen2.dfs.core.windows.net/C7Memberships/{endofmonth}/C7Memberships{endofmonth}.json', jsondata, True)

    